In [49]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [50]:
# Import the expenditure GDP ratio dataset into a Pandas Dataframe
expenditure_ratio_gdp_df = pd.read_csv(
    Path("Resources/expenditure_ratio_gdp.csv"), 
    #index_col='date', 
    #infer_datetime_format=True, 
    #parse_dates=True
)

expenditure_ratio_gdp_df= expenditure_ratio_gdp_df.dropna()
# Review the DataFrame

expenditure_ratio_gdp_df.head()

,Series Name,Series Code,Country Name,Country Code,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
0,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Cabo Verde,CPV,3.074918,3.055903,3.043481,3.174304,3.080391,3.281085,3.209184,3.246215,..,..
1,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Burkina Faso,BFA,1.064500,1.612364,1.626851,1.456188,2.388843,2.610841,2.087066,2.285089,..,..
2,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Cameroon,CMR,0.663232,0.500739,0.613317,0.378912,0.379053,0.145602,0.210445,0.120615,..,..
3,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Canada,CAN,7.401182,7.358208,7.231802,7.580659,7.677841,7.562779,7.582780,7.609296,..,..
4,Domestic general government health expenditure...,SH.XPD.GHED.GD.ZS,Central African Republic,CAF,0.622638,0.982942,0.711088,0.455853,0.599532,0.728822,0.688562,0.819501,..,..


In [51]:
cols=[0,1,2,12,13]
expenditure_ratio_gdp_df.drop(expenditure_ratio_gdp_df.columns[cols], axis=1,inplace=True)

In [52]:
expenditure_ratio_gdp_df

,Country Code,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,CPV,3.074918,3.055903,3.043481,3.174304,3.080391,3.281085,3.209184,3.246215
1,BFA,1.064500,1.612364,1.626851,1.456188,2.388843,2.610841,2.087066,2.285089
2,CMR,0.663232,0.500739,0.613317,0.378912,0.379053,0.145602,0.210445,0.120615
3,CAN,7.401182,7.358208,7.231802,7.580659,7.677841,7.562779,7.582780,7.609296
4,CAF,0.622638,0.982942,0.711088,0.455853,0.599532,0.728822,0.688562,0.819501
5,COG,0.818936,0.936649,0.946950,1.118832,1.204359,0.966202,0.684065,0.783200
6,CIV,0.746079,0.727834,0.777609,0.804460,0.860909,0.949379,0.906541,0.959274
7,DNK,8.600667,8.569009,8.563679,8.609555,8.533851,8.435704,8.447372,8.292176
8,GHA,1.984099,1.870586,1.438298,1.621400,1.327707,1.087230,1.376484,1.375667
9,IND,0.931869,0.865042,0.856535,0.921932,0.940500,0.967347,0.887818,0.988292


In [64]:
expenditure_ratio_gdp_df.rename(columns = {expenditure_ratio_gdp_df.columns[1]:2012, expenditure_ratio_gdp_df.columns[2]:2013,
                                          expenditure_ratio_gdp_df.columns[3]:2014, expenditure_ratio_gdp_df.columns[4]:2015,
                                          expenditure_ratio_gdp_df.columns[5]:2016, expenditure_ratio_gdp_df.columns[6]:2017,
                                          expenditure_ratio_gdp_df.columns[7]:2018, expenditure_ratio_gdp_df.columns[8]:2019}
                                           
                                       , inplace=True)

In [65]:
expenditure_ratio_gdp_df

,Country Code,2012,2013,2014,2015,2016,2017,2018,2019
0,CPV,3.074918,3.055903,3.043481,3.174304,3.080391,3.281085,3.209184,3.246215
1,BFA,1.064500,1.612364,1.626851,1.456188,2.388843,2.610841,2.087066,2.285089
2,CMR,0.663232,0.500739,0.613317,0.378912,0.379053,0.145602,0.210445,0.120615
3,CAN,7.401182,7.358208,7.231802,7.580659,7.677841,7.562779,7.582780,7.609296
4,CAF,0.622638,0.982942,0.711088,0.455853,0.599532,0.728822,0.688562,0.819501
5,COG,0.818936,0.936649,0.946950,1.118832,1.204359,0.966202,0.684065,0.783200
6,CIV,0.746079,0.727834,0.777609,0.804460,0.860909,0.949379,0.906541,0.959274
7,DNK,8.600667,8.569009,8.563679,8.609555,8.533851,8.435704,8.447372,8.292176
8,GHA,1.984099,1.870586,1.438298,1.621400,1.327707,1.087230,1.376484,1.375667
9,IND,0.931869,0.865042,0.856535,0.921932,0.940500,0.967347,0.887818,0.988292


In [66]:
expenditure_ratio_gdp_df.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12
Country Code,CPV,BFA,CMR,CAN,CAF,COG,CIV,DNK,GHA,IND,MLI,SEN,RWA
2012,3.074918,1.0645,0.663232,7.401182,0.622638,0.818936,0.746079,8.600667,1.984099,0.931869,0.413549,1.219237,2.046208
2013,3.055903,1.612364,0.500739,7.358208,0.982942,0.936649,0.727834,8.569009,1.870586,0.865042,0.815838,1.170771,2.163328
2014,3.043481,1.626851,0.613317,7.231802,0.711088,0.94695,0.777609,8.563679,1.438298,0.856535,0.891734,1.080037,2.184646
2015,3.174304,1.456188,0.378912,7.580659,0.455853,1.118832,0.80446,8.609555,1.6214,0.921932,0.919274,1.078207,2.095183
2016,3.080391,2.388843,0.379053,7.677841,0.599532,1.204359,0.860909,8.533851,1.327707,0.9405,1.208768,1.068019,2.222667
2017,3.281085,2.610841,0.145602,7.562779,0.728822,0.966202,0.949379,8.435704,1.08723,0.967347,1.177514,0.956496,2.228853
2018,3.209184,2.087066,0.210445,7.58278,0.688562,0.684065,0.906541,8.447372,1.376484,0.887818,1.151183,0.960297,2.34513
2019,3.246215,2.285089,0.120615,7.609296,0.819501,0.7832,0.959274,8.292176,1.375667,0.988292,1.310267,1.034457,2.56084
